In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from pandas_profiling import ProfileReport
import matplotlib.pyplot as plt 
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

%matplotlib inline
# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Loading Datas

In [ ]:
db = pd.read_csv('/kaggle/input/unsw-nb15/UNSW_NB15_training-set.csv')


# Data set observation

In [ ]:
db.head().T


db['id'] est clairement inutile, supprimons le immédiatement.

In [ ]:
db = db.drop(columns='id')
db.label = db.label.astype('object')


In [ ]:
db.head()

In [ ]:
#pr = ProfileReport(db)

In [ ]:
def descr(bdd):
	for i in [np.number, ['object', 'bool', 'category']]:
		with pd.option_context("display.precision", 2, 'display.float_format', lambda x: '%.3f' % x):
			print(bdd.describe(include=i).T)
			print('\n')
	for i in bdd:
		if bdd[i].isna().sum()>0:
			print("pour {}\t\t il y a {} \t valeurs manquantes".format(i, str(len(bdd[i]) - bdd[i].count())))

descr(db)

Pas de valeurs manquantes.

4. List following information for the training set :

    4.1. Column number
    
    4.2. Column names
    
    4.3. Column types
    
    4.4. Size of the data set


In [ ]:
print (len(db.columns),'\n', db.columns,'\n', db.dtypes,'\n', len(db))

In [ ]:
db.select_dtypes('object').columns

In [ ]:
db.select_dtypes('object').describe()

##### 6. Which columns are categories? List them; extract existing values.


In [ ]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    for i in db.select_dtypes('object').columns:
        print ('for', i)
        print(db[i].value_counts())
        print('#'*30)


#### 7. Which columns are numeric? List them; extract min, max, mean, median and standard deviation values.

In [ ]:
db.select_dtypes(['int', 'float']).describe()

#### 8. Check whether the positive label (1) match attack categories and whether attack categories match labelled data.


In [ ]:
len(db[((db.label == 0) & (db.attack_cat != 'Normal'))]), len(db[((db.label != 0) & (db.attack_cat == 'Normal'))])

=> aucun faux positif ni faux negatif

#### 9. Which is the number of occurrences for each attack category?

cf 6

#### 10. Which protocols and services appear in the positively labelled entries? In the negatively labelled ones?

In [ ]:
db.proto.hist(by=db.label)

In [ ]:
db.service.hist(by=db.label)

In [ ]:
db.proto[db.label==0].value_counts()

In [ ]:
db.proto[db.label==1].value_counts()

D'où : pour les cas non frauduleux on a une vaste majorité de tcp, puis d'udp, avec un petit peu d'arp; dans les cas frauduleux, on a udp avant tcp, et un grand nombre de protocoles plus exotiques.

In [ ]:
db.service[db.label==0].value_counts()

In [ ]:
db.service[db.label==1].value_counts()

On voit aussi sur les services une différence de distribution.

#### 11. What do you conclude about the traffic being analysed?


# Data Visualisation


#### 12. Visualise the repartition of services, protocols, attack types, as histograms


In [ ]:
db.service.value_counts().plot.bar()

In [ ]:
fig = plt.figure(figsize = (35,10))
ax = fig.gca()
db.proto.value_counts().plot.bar()

In [ ]:
# essayons avec les protocoles les plus utilisés

print (float(db.proto.isin(db.proto.value_counts()[db.proto.value_counts()>100].index ).sum()) / len(db))

On représente ainsi 94%+ des données. Représentons les :


In [ ]:
db[db.proto.isin(db.proto.value_counts()[db.proto.value_counts()>100].index )].proto.value_counts().plot.bar()

In [ ]:
db.attack_cat.value_counts().plot.bar()

#### 13. Build the correlation matrix between parameters for labelled and unlabelled entries.


In [ ]:
plt.matshow(db[db.label==0].drop(columns='label').corr())

In [ ]:
plt.matshow(db[db.label==1].drop(columns='label').corr())

In [ ]:
r=db[db.label==0].drop(columns='label').corr()

Ce n'est pas très visuel, même si on voit des corrélations fortes. Essayons d'en lister quelques uns :

In [ ]:
s = db.corr().abs().unstack()
so = s.sort_values(kind="quicksort")

In [ ]:
so[-59:-39]

On a ici les 20 pairs les plus corrélées (les 39 derniers étants à 1, les corrélations de k à k).

On remarque bien logiquement que la moitié sont des doublons.

Voyons sans  : 

In [ ]:
so[-59:-39:2]

Les résultats sont assez logiques : dloss (packet de destination perdus) par rapport à dbytes (bytes émis), pareil pour sloss et sbytes, is_ftp_login et ct_ftp_cmd sont trivialement corrélés etc.  

# Data cleaning

#### 14. Create a new DataFrame X you will use for cleaning.

#### 15. List string fields. Set an integer encoding for all of them.

#### 16. Store the labels in a DataFrame Y

#### 17. Remove labels bound to attacks as well as the ‘id’ field to avoid biased learning

In [ ]:
X = db.drop(columns=['label', 'attack_cat'])
y = db.label
ytype = db.attack_cat
X = pd.get_dummies(X)
y = y.to_numpy().astype('int')

# Machine learning Analysis
#### 18. Through train_test_split on X and Y, and the XGBClassifier
##### 18.1. Extract the AUPRC
##### 18.2. Plot the relative importance of fields through plot_importance, using importance_type ‘cover’, ‘gain’, ‘weight’
##### 18.3. What do you conclude wrt. the security status of the target system?
##### 18.4. Show the classifier visually

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier

X_train, X_test, y_train, y_test = train_test_split(X,y)


In [ ]:
model = GradientBoostingClassifier(n_estimators=200, )
model.fit(X_train, y_train)

In [ ]:
from sklearn.metrics import plot_precision_recall_curve
plot_precision_recall_curve(model, X_test, y_test)

In [ ]:
R=pd.DataFrame (zip(X.columns, model.feature_importances_)).sort_values(1, ascending=False)
R[R[1]>0].plot.bar(x=0, figsize=(35, 5), fontsize=20)

# Statistics Analysis
#### 19. Apply a Principal Component Analysis (PCA) to the dataset X, to extract independent parameters of the model


In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression,LinearRegression
from sklearn.metrics import confusion_matrix

In [ ]:
pca = PCA(0.999999)
stds = StandardScaler()
xt = stds.fit_transform(X_train)
pca.fit(xt)

##### 20. Use Linear Regression to evaluate the contribution of these independent parameters to the characterization as attack

In [ ]:
lr = LinearRegression()
lr.fit(pca.transform(stds.transform(X_train)), y_train)

In [ ]:
# Observons les contributions absolues :
sorted(zip(X_train.columns , lr.coef_), key=lambda x:x[1]**2, reverse=True)

On voit que cela depends particulièrement du service et du protocole, le smtp/sntp ayant l'air assez malveillants.